In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName("spark_task").getOrCreate()

In [3]:
df = spark.read.csv("complete.csv", header=True, inferSchema=True)

In [4]:
df = df.withColumn("Name of State / UT", F.lower(F.col("Name of State / UT")))
df.show()

+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+
|      Date|Name of State / UT|Latitude|Longitude|Total Confirmed cases|Death|Cured/Discharged/Migrated|New cases|New deaths|New recovered|
+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+
|2020-01-30|            kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|
|2020-01-31|            kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|
|2020-02-01|            kerala| 10.8505|  76.2711|                  2.0|    0|                      0.0|        1|         0|            0|
|2020-02-02|            kerala| 10.8505|  76.2711|                  3.0|    0|                      0.0|        1|         0|            0|
|2020-02-03|        

In [5]:
max_cases = df.orderBy(F.col("New cases").desc()).first()
print(max_cases["Date"])

2020-07-26


In [6]:
df_grouped = df.groupBy("Name of State / UT").sum("Total Confirmed cases")
second_largest_state = df_grouped.orderBy(F.col("sum(Total Confirmed cases)").desc()).take(2)[1]
print(second_largest_state['Name of State / UT'])

tamil nadu


In [7]:
df_ut = df.filter(F.col("Name of State / UT").contains("union territory of"))
df_ut = df_ut.withColumn("Death", F.col("Death").cast("int"))
deaths = df_ut.groupBy("Name of State / UT").sum("Death")
least_deaths = deaths.orderBy(F.col("sum(Death)").asc()).first()
print(least_deaths['Name of State / UT'])

union territory of ladakh


In [8]:
df = df.withColumn("Death to Confirmed Ratio", F.col("Death") / F.col("Total Confirmed cases"))
state_lowest_ratio = df.orderBy(F.col("Death to Confirmed Ratio").asc()).first()
print(state_lowest_ratio['Name of State / UT'])

puducherry


In [9]:
month_map = {
    1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June",
    7: "July", 8: "August", 9: "September", 10: "October", 11: "November", 12: "December"
}

df = df.withColumn("Month", F.month(F.col("Date")))
new_recovered = df.groupBy("Month").sum("New recovered").orderBy(F.col("sum(New recovered)").desc()) .first()
print(month_map[new_recovered['Month']])

July
